In [764]:
# import libraries
import numpy as np
import pandas as pd
import math
import statistics as st
from scipy.stats import zscore

In [765]:
# define a function to convert the columns of a df into all lower case
def convert_cols_to_lower(df):
    df.columns = [ i.lower() for i in df.columns ]
    return df

In [766]:
# define a column list with the names for all the columns
column_lst = sorted(['Customer', 'State', 'Gender', 'Education', 'Customer Lifetime Value',
       'Income', 'Monthly Premium Auto', 'Number of Open Complaints',
       'Policy Type', 'Vehicle Class', 'Total Claim Amount'])

# define a function to read all three dataset into three files, assure the column names are the same and then concatenate them into one dataset
def load_original_data():
    file1 = pd.read_csv('Data/file1.csv')
    file2 = pd.read_csv('Data/file2.csv')
    file3 = pd.read_csv('Data/file3.csv')
    
    convert_cols_to_lower(file1)
    convert_cols_to_lower(file2)
    convert_cols_to_lower(file3)
    
    file1 = file1.sort_index(axis=1)
    file2 = file2.sort_index(axis=1)
    file3 = file3.sort_index(axis=1)
    
    file1.columns = column_lst
    file2.columns = column_lst
    # replace state by st in file3
    file3.columns = file3.columns.str.replace("state", "st")
    file3.columns = column_lst
    
    df = pd.concat([file1, file2, file3], axis=0)
    
    return df

In [768]:
# load the full dataset, correctly concatenated and in the desired order
full_dataset = load_original_data()

In [769]:
# turn column names into lower and replace space by underscore
full_dataset.columns = [ i for i in full_dataset.columns.str.replace(" ","_").str.lower()]

In [771]:
# drop Customer column as it is basically an index
full_dataset = full_dataset.drop("customer", axis=1)

In [772]:
# remove duplicates
full_dataset = full_dataset.drop_duplicates()

In [774]:
# reset the index to match the amount of rows
full_dataset = full_dataset.reset_index()

In [775]:
# remove "%" signs from Customer Lifetime Value where present and divide these values by 100 to account for unusually high values probably caused by wrong formatting and therefore multiplying by 100
full_dataset["customer_lifetime_value"] = (
    full_dataset["customer_lifetime_value"].apply(lambda x: float(x.replace("%", " "))/100 if type(x) == str else x))

In [776]:
# convert Customer Lifetime Value into integer by converting it to numeric and then truncating it (as requested in assignment task)
full_dataset["customer_lifetime_value"] = pd.to_numeric(full_dataset["customer_lifetime_value"], errors = "coerce")
# using lambda function to check for nans and if not nan apply truncate
full_dataset["customer_lifetime_value"] = full_dataset["customer_lifetime_value"].apply(lambda x: math.trunc(x) if np.isnan(x) != True else 0)

In [778]:
# function that checks if input is a string and splits it at "/" if it is
# returning the 2nd element of the resulting list or returning x if it's not a string
def split(x):
    if isinstance(x, str):
        return x.split("/")[1]
    return x

In [779]:
# apply split() and convert to numbers
full_dataset["number_of_open_complaints"] = full_dataset["number_of_open_complaints"].apply(lambda x: split(x))
full_dataset["number_of_open_complaints"] = pd.to_numeric(full_dataset["number_of_open_complaints"], errors = "coerce")

In [780]:
# check for nans
full_dataset.loc[full_dataset["number_of_open_complaints"].isna()]

,index,customer_lifetime_value,education,gender,income,monthly_premium_auto,number_of_open_complaints,policy_type,state,total_claim_amount,vehicle_class
1070,1071,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [781]:
# drop row 1071 that only contains nans
full_dataset = full_dataset.drop([1071])

In [782]:
# function that checks if a given string is found in a list of possible descriptors of male and female and then replaces them for the same
def clean_gender(string):
    m_lst = ['M', 'Male']
    f_lst = ['F', 'Femal', 'female']
    if isinstance(string, str) and string in m_lst:
        return "male"
    elif isinstance(string, str) and string in f_lst:
        return "female"
    else:
        return "error"

In [783]:
# apply the map function to generate the "gender" column again and overwrite the old column with it
full_dataset['gender'] = list(map(clean_gender, full_dataset['gender']))

## Day 2

In [785]:
# replace 0s with np.nan
full_dataset["income"] = full_dataset["income"].replace(to_replace=0, value=np.nan)

In [786]:
# fill mean of column to elements with NaN
full_dataset["income"] = full_dataset["income"].replace(to_replace=np.nan, value=full_dataset["income"].mean())

In [787]:
# replace 0s with np.nan
full_dataset["monthly_premium_auto"] = full_dataset["monthly_premium_auto"].replace(to_replace=0, value=np.nan)
# fill mean of column to elements with NaN
full_dataset["monthly_premium_auto"] = (
    full_dataset["monthly_premium_auto"].replace(to_replace=np.nan, value=full_dataset["monthly_premium_auto"].mean()))

In [788]:
# replace 0s with np.nan
full_dataset["total_claim_amount"] = full_dataset["total_claim_amount"].replace(to_replace=0, value=np.nan)
# fill mean of column to elements with NaN
full_dataset["total_claim_amount"] = (
    full_dataset["total_claim_amount"].replace(to_replace=np.nan, value=full_dataset["total_claim_amount"].mean()))

Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central

In [770]:
# check for values in column "state"
full_dataset["state"].unique()

array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon', 'Cali',
       'AZ', 'WA', nan], dtype=object)

In [789]:
region_lst = ['California', 'Cali', 'Oregon', "Washington", 'WA', "Arizona", 'AZ', "Nevada"]
def bucket(string, lst):
    if string == (lst[0] or lst[1]):
        return "west_region"
    elif string == lst[2]:
        return "north_west"
    elif string == (lst[3] or lst[4]):
        return "east"
    elif string == (lst[5] or lst[6] or lst[7]):
        return "central"
    else:
        return np.nan

In [790]:
full_dataset["state"] = full_dataset["state"].apply(lambda x: bucket(x,region_lst))

In [791]:
full_dataset = full_dataset.drop("index", axis=1)

In [793]:
full_dataset["education"] = full_dataset["education"].str.lower()
full_dataset["policy_type"] = full_dataset["policy_type"].str.lower()
full_dataset["vehicle_class"] = full_dataset["vehicle_class"].str.lower()

In [796]:
replace_dict = {"luxury suv": "luxury vehicle" , "luxury car" : "luxury vehicle"}
full_dataset["vehicle_class"] = full_dataset["vehicle_class"].replace(replace_dict)

In [797]:
# create index of all columns with numerical data
numeric_cols = full_dataset.select_dtypes(include=[np.number]).columns
# delete the ones that dont make any sense to remove the outliers from
numeric_cols = numeric_cols.drop(["number_of_open_complaints"])

In [799]:
# create a dataframe only containing numerical data, based on the index numeric_cols and then apply the zscore to filter out the outliers above a threshold of three with is comparable to the 1.5*IQR method
numeric_data = full_dataset[numeric_cols]
z = full_dataset[numeric_cols].apply(zscore)
threshold = 3

# filter the dataframe to remove the outliers
full_dataset = full_dataset[(z < threshold).all(axis=1)]